In [31]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import os
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'avaiable device : %s' % device)
# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

avaiable device : cuda


In [17]:
json_path = 'train'
path = "all_data_Noncrop"
path_target = "all_data_Noncrop"

In [18]:
import json
import pandas as pd
x = classes = ['ang','anx','emb','happy','normal','pain','sad']
x = classes = ['angry','anxiety','embarrass','happy','normal','pain','sad']

# JSON 파일 읽기
rows = []
for i in x:
    with open(f'data/{json_path}_all_label/{i}.json', 'r') as file:
        data = json.load(file)
###########################

    for item in data:
        row = {
            'filename': item['filename'],
            'gender': item['gender'],
            'age': item['age'],
            'isProf': item['isProf'],
            'faceExp_uploader': item['faceExp_uploader'],
            'bg_uploader': item['bg_uploader'],
            'annot_A_maxX': item['annot_A']['boxes']['maxX'],
            'annot_A_maxY': item['annot_A']['boxes']['maxY'],
            'annot_A_minX': item['annot_A']['boxes']['minX'],
            'annot_A_minY': item['annot_A']['boxes']['minY'],
            'annot_A_faceExp': item['annot_A']['faceExp'],
            'annot_A_bg': item['annot_A']['bg'],
            'annot_B_maxX': item['annot_B']['boxes']['maxX'],
            'annot_B_maxY': item['annot_B']['boxes']['maxY'],
            'annot_B_minX': item['annot_B']['boxes']['minX'],
            'annot_B_minY': item['annot_B']['boxes']['minY'],
            'annot_B_faceExp': item['annot_B']['faceExp'],
            'annot_B_bg': item['annot_B']['bg'],
            'annot_C_maxX': item['annot_C']['boxes']['maxX'],
            'annot_C_maxY': item['annot_C']['boxes']['maxY'],
            'annot_C_minX': item['annot_C']['boxes']['minX'],
            'annot_C_minY': item['annot_C']['boxes']['minY'],
            'annot_C_faceExp': item['annot_C']['faceExp'],
            'annot_C_bg': item['annot_C']['bg']
        }
        rows.append(row)

# 데이터프레임 생성
df = pd.DataFrame(rows)

# 데이터프레임 확인
df.head()


,filename,gender,age,isProf,faceExp_uploader,bg_uploader,annot_A_maxX,annot_A_maxY,annot_A_minX,annot_A_minY,...,annot_B_minX,annot_B_minY,annot_B_faceExp,annot_B_bg,annot_C_maxX,annot_C_maxY,annot_C_minX,annot_C_minY,annot_C_faceExp,annot_C_bg
0,ed8e05a7770502606d8278a279c624e4fb7e681fd09766...,여,30,전문인,분노,상업시설/점포/시장,1780.973600,1379.698500,957.011300,304.805400,...,978.322424,335.616310,상처,행사/사무공간,1765.113605,1393.053009,975.533139,339.338318,분노,행사/사무공간
1,6ea444752308b9fc4ce87b2dc63c1341da53b94bc7804a...,여,20,전문인,분노,도심 환경,1813.149164,1698.626008,1006.772046,652.801981,...,985.222270,618.989536,분노,공공시설/종교/의료,1858.686000,1780.722300,979.551000,610.389900,분노,숙박 및 거주공간
2,ca3a4e6e725cbd4765f642703c442507648cefb3eb9262...,여,20,전문인,분노,상업시설/점포/시장,1289.506949,1808.273872,365.400991,427.611043,...,367.927260,381.171959,불안,상업시설/점포/시장,1288.759354,1801.691047,365.415146,433.177002,분노,상업시설/점포/시장
3,aec709ad22845bdbb7103c9e79cede5ac316fd4a3dc0e4...,여,20,전문인,분노,실외 자연환경,1451.527488,1697.697719,620.942293,588.667819,...,616.706900,605.596300,분노,도심 환경,1460.023738,1714.676738,628.042463,580.202562,분노,공공시설/종교/의료
4,bc4c9e3eeb74e4d173f26b3a184f41f545c76e7f3a21d2...,여,20,전문인,분노,실외 자연환경,2122.442811,1686.675987,1377.577404,571.080934,...,1402.146230,613.606782,분노,실외 자연환경,2120.577934,1664.955234,1367.480034,602.995534,분노,실외 자연환경


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 417167 entries, 0 to 417166
Data columns (total 24 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   filename          417167 non-null  object 
 1   gender            417167 non-null  object 
 2   age               417167 non-null  int64  
 3   isProf            417167 non-null  object 
 4   faceExp_uploader  417167 non-null  object 
 5   bg_uploader       417167 non-null  object 
 6   annot_A_maxX      417167 non-null  float64
 7   annot_A_maxY      417167 non-null  float64
 8   annot_A_minX      417167 non-null  float64
 9   annot_A_minY      417167 non-null  float64
 10  annot_A_faceExp   417167 non-null  object 
 11  annot_A_bg        417167 non-null  object 
 12  annot_B_maxX      417167 non-null  float64
 13  annot_B_maxY      417167 non-null  float64
 14  annot_B_minX      417167 non-null  float64
 15  annot_B_minY      417167 non-null  float64
 16  annot_B_faceExp   41

# YOLOv8 형식에 맞게끔 포맷 칼럼 추가

In [20]:

# 바운딩 박스의 중심 좌표와 너비, 높이를 계산합니다.
df['x_center'] = df['annot_A_minX'] + (df['annot_A_maxX'] - df['annot_A_minX']) / 2
df['y_center'] = df['annot_A_minY'] + (df['annot_A_maxY'] - df['annot_A_minY']) / 2
df['w'] = df['annot_A_maxX'] - df['annot_A_minX']
df['h'] = df['annot_A_maxY'] - df['annot_A_minY']



정규화 아직안함

---

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 417167 entries, 0 to 417166
Data columns (total 28 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   filename          417167 non-null  object 
 1   gender            417167 non-null  object 
 2   age               417167 non-null  int64  
 3   isProf            417167 non-null  object 
 4   faceExp_uploader  417167 non-null  object 
 5   bg_uploader       417167 non-null  object 
 6   annot_A_maxX      417167 non-null  float64
 7   annot_A_maxY      417167 non-null  float64
 8   annot_A_minX      417167 non-null  float64
 9   annot_A_minY      417167 non-null  float64
 10  annot_A_faceExp   417167 non-null  object 
 11  annot_A_bg        417167 non-null  object 
 12  annot_B_maxX      417167 non-null  float64
 13  annot_B_maxY      417167 non-null  float64
 14  annot_B_minX      417167 non-null  float64
 15  annot_B_minY      417167 non-null  float64
 16  annot_B_faceExp   41

In [23]:
# 클래스 목록을 지정합니다
classes = ['ang','anx','emb','happy','normal','pain','sad']
classes2 = ['분노','불안','당황','행복','중립','상처','슬픔']
classes3 = ['ang','anx','emb','hap','nor','pai','sad']  # classes2와 매칭되는 클래스 이름입니다
import os
import glob
import pandas as pd
# classes = ['ang']
# 빈 리스트를 생성합니다
data = []

# 각 클래스에 대해
for cls, cls2 in zip(classes3, classes2):  # 클래스와 클래스2를 함께 순회합니다
    # 임시 경로 추가
    # cls = f"data/{path_target}/images"
    cls = f"data/all_data_Noncrop/{cls}"
    # 경로 알아서 수정
    

    # 클래스 폴더 내의 모든 .jpg 파일을 가져옵니다
    # files = glob.glob(os.path.join(cls, '*.jpg'))
    files = glob.glob(os.path.join(cls, '*.jpg')) + glob.glob(os.path.join(cls, '*.jpeg'))


    for filename in files:
        # 파일 이름과 클래스를 데이터프레임에 추가합니다
        # df = df.append({'filename': filename, 'class': cls}, ignore_index=True)
        # print(filename)

        base_filename = os.path.basename(filename)
        
        data.append({'filename': base_filename, 'class': cls2})  # 클래스를 cls2로 변경하여 넣습니다
    
    # break# 한번만돌기위함

# 리스트를 데이터프레임으로 변환합니다
df2 = pd.DataFrame(data)


In [34]:
import os
import glob
import pandas as pd

# 클래스 목록을 지정합니다
classes = ['ang','anx','emb','happy','normal','pain','sad']
classes2 = ['분노','불안','당황','행복','중립','상처','슬픔']
classes3 = ['ang','anx','emb','hap','nor','pai','sad']

# classes = ['ang']
# 빈 리스트를 생성합니다
data = []

# 각 클래스에 대해
for cls in classes3:
    # 임시 경로 추가
    # cls = f"data/{path_target}/images"
    cls = f"data/train/{cls}_crop_landmark"
    # 경로 알아서 수정
    

    # 클래스 폴더 내의 모든 .jpg 파일을 가져옵니다
    # files = glob.glob(os.path.join(cls, '*.jpg'))
    files = glob.glob(os.path.join(cls, '*.jpg')) + glob.glob(os.path.join(cls, '*.jpeg'))


    for filename in files:
        # 파일 이름과 클래스를 데이터프레임에 추가합니다
        # df = df.append({'filename': filename, 'class': cls}, ignore_index=True)
        # print(filename)

        base_filename = os.path.basename(filename)
        
        data.append({'filename': base_filename, 'class': cls})
    
    # break# 한번만돌기위함

# 리스트를 데이터프레임으로 변환합니다
df2 = pd.DataFrame(data)

# 데이터프레임을 출력합니다
# print(df2)

In [35]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14000 entries, 0 to 13999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  14000 non-null  object
 1   class     14000 non-null  object
dtypes: object(2)
memory usage: 218.9+ KB


In [24]:
# df1과 df2를 'filename' 열을 기준으로 병합합니다
merged_df = pd.merge(df2, df, on='filename')

# 병합된 데이터프레임을 출력합니다
print(merged_df)


                                                 filename class gender  age  \
0       06e5f8f8123ee831d958c5c7b99f48f08251d756922cda...    분노      남   30   
1       5b77f18724833ccd71967b89d9152a588ed584df00d939...    분노      여   20   
2       0d18ae1ed7d9e912864b736523a94a24db0595bf273964...    분노      여   30   
3       5a30dc193e2c6144184968043c00773270b111a23a3c83...    분노      남   30   
4       007f299267a9d6a145f627e9993aeae4e8323652785a2f...    분노      남   30   
...                                                   ...   ...    ...  ...   
112698  2abe61df49f61d753287e56b12f174d63df7ccc08f8ea0...    슬픔      남   40   
112699  4a889bb2caf94fef2266933b7ff17b34a746ea5074036e...    슬픔      남   20   
112700  5c2261c629f9f20130b1127d24231c162a7b62ff959abe...    슬픔      여   20   
112701  4a889bb2caf94fef2266933b7ff17b34a746ea5074036e...    슬픔      남   20   
112702  4a889bb2caf94fef2266933b7ff17b34a746ea5074036e...    슬픔      남   20   

       isProf faceExp_uploader        bg_uploader  

In [25]:
count = 0
for index, row in merged_df.iterrows():
    if row['class'] != row['annot_A_faceExp'] and row['class'] != row['annot_B_faceExp'] and row['class'] != row['annot_C_faceExp']:
        count += 1


In [29]:
print(count)

32681


# 이미지 정규화 시도 후 칼럼추가


In [ ]:
import pandas as pd
from PIL import Image
import os

# 이미지 파일이 저장된 디렉토리
image_dir = f'data/{path_target}/images'

# 각 이미지의 너비와 높이로 정규화합니다.
for i, row in merged_df.iterrows():
    # 이미지 파일의 경로
    image_path = os.path.join(image_dir, row['filename'])

    # 이미지 열기
    img = Image.open(image_path)

    # 이미지의 너비와 높이 얻기
    img_width, img_height = img.size

    # 정규화
    # 새로운 열을 추가합니다.
    merged_df['N_x_center'] = merged_df['x_center'] / img_width
    merged_df['N_y_center'] = merged_df['y_center'] / img_height
    merged_df['N_w'] = merged_df['w'] / img_width
    merged_df['N_h'] = merged_df['h'] / img_height


# 이미지 정규화말고 predict값으로 값 집어넣기

In [ ]:
import pandas as pd
from PIL import Image
import os
from ultralytics import YOLO
import cv2

# 이미지 파일이 저장된 디렉토리
image_dir = f'data/{path_target}/images'
# YOLO 모델을 지정된 가중치 파일로 로드합니다
model = YOLO("yolov8l-face.pt")

# 각 이미지의 너비와 높이로 정규화합니다.
for i, row in merged_df.iterrows():
    # 이미지 파일의 경로
    image_path = os.path.join(image_dir, row['filename'])
    # 이미지 열기
    img = Image.open(image_path)
    
    results = model(image_path)
    xywhn = results[0].boxes.xywhn[0]
    N_x_center, N_y_center, N_w, N_h = xywhn.cpu().numpy()

    # 정규화
    # 새로운 열을 추가합니다.
    merged_df['N_x_center'] = N_x_center
    merged_df['N_y_center'] = N_y_center
    merged_df['N_w'] = N_w
    merged_df['N_h'] = N_h


In [30]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112703 entries, 0 to 112702
Data columns (total 29 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   filename          112703 non-null  object 
 1   class             112703 non-null  object 
 2   gender            112703 non-null  object 
 3   age               112703 non-null  int64  
 4   isProf            112703 non-null  object 
 5   faceExp_uploader  112703 non-null  object 
 6   bg_uploader       112703 non-null  object 
 7   annot_A_maxX      112703 non-null  float64
 8   annot_A_maxY      112703 non-null  float64
 9   annot_A_minX      112703 non-null  float64
 10  annot_A_minY      112703 non-null  float64
 11  annot_A_faceExp   112703 non-null  object 
 12  annot_A_bg        112703 non-null  object 
 13  annot_B_maxX      112703 non-null  float64
 14  annot_B_maxY      112703 non-null  float64
 15  annot_B_minX      112703 non-null  float64
 16  annot_B_minY      11

In [ ]:
classes = ['분노','불안','당황','기쁨','중립','상처','슬픔']
class_to_index = {cls: idx for idx, cls in enumerate(classes)}
print(class_to_index)

In [ ]:
# 각 파일에 대해 별도의 .txt 파일을 생성합니다.
for filename in merged_df['filename'].unique():
    # 확장자를 제거합니다.
    filename_without_ext = os.path.splitext(filename)[0]
    # 특정 파일에 대한 모든 행을 선택합니다.
    file_df = merged_df[merged_df['filename'] == filename]
    
    # .txt 파일을 쓰기 모드로 엽니다.
    with open(os.path.join(f'data/{path_target}/labels', f'{filename_without_ext}.txt'), 'w') as f:
        # 각 행에 대해 클래스와 좌표를 파일에 씁니다.
        for idx, row in file_df.iterrows():
            # 클래스를 인덱스로 변환합니다.
            class_index = class_to_index[row['faceExp_uploader']]
            # 클래스 인덱스와 좌표를 띄어쓰기로 구분하여 저장합니다.



            #이게 yolo8 포맷 정규화 형식으로 넣는것
            f.write(f"{class_index} {row['N_x_center']} {row['N_y_center']} {row['N_w']} {row['N_h']}\n")
            #이게 yolo8 포맷 형식으로 넣는것
            # f.write(f"{class_index} {row['x_center']} {row['y_center']} {row['w']} {row['h']}\n")
            # f.write(f"{class_index} {row['annot_A_maxX']} {row['annot_A_maxY']} {row['annot_A_minX']} {row['annot_A_minY']}\n")


# 폴더별로 나뉘어진 파일 전부 상위폴더로 이동시키는 코드

In [ ]:
import os
import glob
import shutil

# 클래스 목록을 지정합니다
classes = ['ang','anx','emb','hap','nor','pai','sad']

# 이미지가 저장된 디렉토리를 지정합니다. 이 경로는 현재 작업 디렉토리를 기준으로 합니다.
image_dir = 'data/train/images'

# 각 클래스에 대해
for cls in classes:
    # 클래스 폴더 내의 모든 .jpg와 .jpeg 파일을 가져옵니다
    image_files = glob.glob(os.path.join(image_dir, cls, '*.jpg')) + glob.glob(os.path.join(image_dir, cls, '*.jpeg'))

    # 각 이미지 파일을 상위 디렉토리로 이동합니다
    for filename in image_files:
        shutil.move(filename, os.path.join(image_dir, os.path.basename(filename)))


# 데이터가 한개만 있다면 train validation 나누는 코드

In [ ]:
import os
import glob
import numpy as np
import shutil

# 클래스 목록을 지정합니다
classes = ['ang','anx','emb','happy','normal','pain','sad']

# 이미지와 레이블이 저장된 디렉토리를 지정합니다
image_dir = 'data/train/images'
# label_dir = '../../label'

# validation 파일이 저장될 디렉토리를 지정합니다
val_image_dir = 'data/validation/images'
# val_label_dir = 'data/validation/labels'

# validation 디렉토리가 없으면 생성합니다
os.makedirs(val_image_dir, exist_ok=True)
# os.makedirs(val_label_dir, exist_ok=True)

# 각 클래스에 대해
for cls in classes:
    # 임시
    cls = ""

    # 클래스 폴더 내의 모든 .jpg와 .jpeg 파일을 가져옵니다
    image_files = glob.glob(os.path.join(image_dir, cls, '*.jpg')) + glob.glob(os.path.join(image_dir, cls, '*.jpeg'))

    # 이미지 파일 목록을 무작위로 섞습니다
    np.random.shuffle(image_files)
    
    # 이미지 파일의 10%를 validation set으로 사용합니다
    val_size = int(len(image_files) * 0.1)

    # validation set에 해당하는 이미지 파일을 이동합니다
    for filename in image_files[:val_size]:
        # 이미지 파일을 이동합니다
        shutil.move(filename, os.path.join(val_image_dir, os.path.basename(filename)))

        # 동일한 이름의 레이블 파일을 이동합니다

        # label_filename = os.path.splitext(os.path.basename(filename))[0] + '.txt'
        # shutil.move(os.path.join(label_dir, label_filename), os.path.join(val_label_dir, label_filename))


# 안맞는 fair찾기

In [ ]:
import os

# 이미지와 레이블 디렉토리를 지정합니다
# image_dir = 'train/images'
# label_dir = 'train/label'

# image_dir = 'validation/images'
# label_dir = 'validation/labels'

image_dir = 'test/images'
label_dir = 'test/label'

# 이미지와 레이블 파일의 이름을 가져옵니다
image_files = set(os.path.splitext(file)[0] for file in os.listdir(image_dir))
label_files = set(os.path.splitext(file)[0] for file in os.listdir(label_dir))

# 이미지 파일과 레이블 파일의 차집합을 구합니다
missing_labels = image_files - label_files
missing_images = label_files - image_files

# 누락된 레이블 파일을 출력합니다
for file in missing_labels:
    print(f'Missing label file for image: {file}.jpg')
    # os.remove(os.path.join(label_dir, f'{file}.txt'))

# 누락된 이미지 파일을 출력합니다
for file in missing_images:
    print(f'Missing image file for label: {file}.txt')
    # os.remove(os.path.join(image_dir, f'{file}.jpg'))
